In [1]:
import torch
import torchvision
import glob
from PIL import Image
import matplotlib.pyplot as plt
import random
import os

/usr/local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
ANSWER_SIZE = 128
WINDOW_SIZE = int(ANSWER_SIZE / 2)
MINI_BATCH = 10
EPOCH = 500

In [3]:
files = glob.glob('./dataset/images/*')
train_files = files[0:20]
valid_files = files[20:30]
print('{} images loaded.(train:valid={}:{})'.format(len(files),len(train_files),len(valid_files)))

30 images loaded.(train:valid=20:10)


In [4]:
class UpSampleNet(torch.nn.Module):
    def __init__(self):
        super(UpSampleNet, self).__init__()
        self.input = self.output = 0
        self.layer1 = self.layer2 = self.layer3 = self.layer4 = self.layer5 = 0
        
        self.step1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU(),
        )

        self.step2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2,stride=2),
            torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(64), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(64), torch.nn.ReLU(),
        )

        self.step3 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2,stride=2),
            torch.nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(128), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(128), torch.nn.ReLU(),
        )

        self.deconv1 = torch.nn.ConvTranspose2d(in_channels=128,out_channels=64,kernel_size=2,stride=2)

        self.step4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=128,out_channels=64,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(64), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(64), torch.nn.ReLU(),
        )

        self.deconv2 = torch.nn.ConvTranspose2d(in_channels=64,out_channels=32,kernel_size=2,stride=2)

        self.step5 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(32), torch.nn.ReLU(),
        )

        self.deconv3 = torch.nn.ConvTranspose2d(in_channels=32,out_channels=16,kernel_size=2,stride=2)

        self.step6 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16,out_channels=8,kernel_size=3,padding=1),
            torch.nn.BatchNorm2d(8), torch.nn.ReLU(),
            torch.nn.Conv2d(in_channels=8,out_channels=3,kernel_size=3,padding=1),
            torch.nn.Sigmoid()
        )

    def forward(self, small):
        self.input = small
        # Encode
        self.layer1 = self.step1(self.input)
        self.layer2 = self.step2(self.layer1)
        self.layer3 = self.step3(self.layer2)
        # Decode
        self.layer3 = self.deconv1(self.layer3)
        self.layer4 = self.step4(torch.cat((self.layer3,self.layer2),dim=1))
        self.layer4 = self.deconv2(self.layer4)
        self.layer5 = self.step5(torch.cat((self.layer4,self.layer1),dim=1))
        self.layer5 = self.deconv3(self.layer5)
        self.output = self.step6(self.layer5)
        return self.output


In [5]:
def infer(model, small, padding=WINDOW_SIZE, stride=WINDOW_SIZE):
    model.train(False)
    h,w = len(small[0]), len(small[0,0])
    small = torchvision.transforms.functional.pad(small, padding)
    large = torch.zeros(len(small),len(small[0])*2,len(small[0,0])*2)

    top = 0
    while top + WINDOW_SIZE <= len(small[0]):
        left = 0
        while left + WINDOW_SIZE <= len(small[0,0]):
            input = torchvision.transforms.functional.crop(small,top,left,WINDOW_SIZE,WINDOW_SIZE)
            large[:,top*2:top*2+ANSWER_SIZE,left*2:left*2+ANSWER_SIZE] = model(torch.unsqueeze(input,dim=0))
            left += stride
        top += stride
    
    large = torchvision.transforms.functional.center_crop(large,[h*2,w*2])
    return torch.squeeze(large)

In [6]:
net = UpSampleNet()
net.load_state_dict(torch.load('./outputs/best_model.pth'))

<All keys matched successfully>

In [7]:
for filename in valid_files:
    image = Image.open(filename)
    image = torchvision.transforms.functional.to_tensor(image)
    
    large_image = infer(net, image)
    #torchvision.utils.save_image(large_image, './outputs/infer/'+filename.split('/')[-1])

: 

: 